<h1 align="center">TAGx AI Day</h1>
<h2 align="center">Custom Vision - Cognitive services Lab 2</h2>
<h2 align="center">Object Detection</h2>

## Setting Up Environment

1. Go to https://customvision.ai/  and sign in with your azure account/ID
2. Once you are in, go to Settings (gear icon - upper right corner) and copy your Limited Trial Prediction and Training Key

## Initialize the libraries and origin and destination workspaces

In [ ]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry, Region

import numpy as np
import cv2
from matplotlib import pyplot as plt
import urllib.request as ul

# DESTINATION Resource Group Keys
dest_training_key = "<TRAINING KEY>"
dest_prediction_key = "<PREDICTION KEY>"
dest_project_new_name = "PowerItems_Detection_Copy"

# ORIGIN Resource Group Keys
training_key = "970bf90c200d4f429a233dee4239e8f1"
prediction_key = "23287e52d75349138403089743a504ab"

trainer = training_api.TrainingApi(training_key)
dest_trainer = training_api.TrainingApi(dest_training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")
dest_obj_detection_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "ObjectDetection")

## Get the origin project ID reference

In [ ]:
myProjects = trainer.get_projects()

In [ ]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

In [ ]:
Project = trainer.get_project(project_id="70e7718a-e1f1-45d5-bba1-60b224f55365")

## Create the destination Project 

In [ ]:
dest_Project = dest_trainer.create_project(dest_project_new_name, domain_id=dest_obj_detection_domain.id)

In [ ]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

## Get the tags on origin project and create same tags on destination project

In [ ]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

In [ ]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

## Get the images with regions on origin project

In [ ]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [ ]:
tagged_images_with_regions = []
for image in tagged_images:
    dest_regions = []
    dest_tags_ids = []
    
    for tag in image.tags: #for each tag on the origin image
        dest_tags_ids.append(dest_tags_dict[tag.tag_name]) 
    for region in image.regions: #for each region on the origin image
        dest_regions.append(Region(tag_id=dest_tags_dict[region.tag_name], left=region.left, top=region.top, width=region.width, height=region.height))
    
    tagged_images_with_regions.append(ImageUrlCreateEntry(url=image.image_uri, tag_ids=dest_tags_ids, regions=dest_regions))
print("Done")

## Create the images with regions on destination project

In [ ]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_regions)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_regions[begin:end])

In [ ]:
print("Count of Tagged images on origin project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

## Train the model on destination Project

In [ ]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

## Get performance metrics

In [ ]:
performance = dest_trainer.get_iteration_performance(dest_Project.id, iteration.id, threshold=0.15, overlap_threshold=0.30)
print("Iteration: " + iteration.name)
print("Precision: " + str(performance.precision * 100)+ "%")
print("Recall: " + str(performance.recall * 100) + "%")

In [ ]:
per_tag_performance = performance.per_tag_performance

In [ ]:
for tag in per_tag_performance:
    print ("Tag: " + tag.name + "\t\t" + "Precision: " + str(tag.precision) + "\t" + "Recall: " + str(tag.recall))

## Get and use the default prediction endpoint

In [ ]:
# Select an image to run through the model. Feel free to try different images!

url = 'https://cnx.org/resources/0dda2e5d58aedea82e70669236f6503c938f58a4/CNX_UPhysics_32_06_Twork.jpg'

data = None
try:
    data = ul.urlopen(url).read()
except Exception as e:
    print("Could not download the image: %s " % e.message)
else:
    data =  np.frombuffer(data, np.uint8)
    img_data = cv2.imdecode(data, cv2.IMREAD_COLOR)
    img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
    plt.rcParams["figure.figsize"] = [16,9]
    plt.imshow(img_data)
    plt.show()

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = prediction_endpoint.PredictionEndpoint(dest_prediction_key)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [ ]:
from PIL import Image, ImageDraw
from urllib.request import urlopen
import io
from IPython.display import display

file = urlopen(url)
image_file = io.BytesIO(file.read())
im = Image.open(image_file)
x,y = im.size
draw = ImageDraw.Draw(im)

# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.25): # only show the prediction with more than 25% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100), "   Box relative points:",prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)
        x0 = x * prediction.bounding_box.left
        y0 = y * prediction.bounding_box.top
        x1 = x0 + x * prediction.bounding_box.width
        y1 = y0 + y * prediction.bounding_box.height
        draw.rectangle(((x0 , y0 ), (x1 , y1)), outline="#ff8888")
        draw.text((x0, y0-10), prediction.tag_name)

display(im)